<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/modelToDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##モデルをDB化

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import IPython
#bz2ファイル解凍
!bunzip2 -k -q "./drive/My Drive/NLP/20170201.tar.bz2"

#tarファイル解凍
!tar -xvf "./drive/My Drive/NLP/20170201.tar"

bunzip2: Output file ./drive/My Drive/NLP/20170201.tar already exists.
entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


In [41]:
from gensim.models import KeyedVectors

#解凍時のみ
model_dir = 'entity_vector/entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True,limit=500000)

#解凍後確認
results = model.most_similar(positive=[u'[坂井泉水]',u'逆'])
for result in results:
  print(result)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


('[大黒摩季]', 0.551082193851471)
('[来生えつこ]', 0.5425988435745239)
('[岸田繁]', 0.536811351776123)
('[田島貴男]', 0.535373330116272)
('原曲', 0.5254271626472473)
('[ZARD]', 0.5241447687149048)
('[宇徳敬子]', 0.5184545516967773)
('[松本隆]', 0.5148824453353882)
('[宝野アリカ]', 0.5131203532218933)
('[栗林誠一郎]', 0.5111038088798523)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [30]:
#、。「」(括弧)、数字などの記号は変換しない方針とする
cnt = 0

for k, v in model.vocab.items():
  k = delete_brackets(k)

  #数値を含まない単語が対象
  if re.match('.*?(\d+).*', k) is None:
    print(k)
    cnt = cnt + 1

    if cnt > 100:
      break


の
、
。
に
は
を
た
が
で
て
と
し
（
れ
さ
）
ある
いる
する
も
「
・
」
から
こと
な
として
い
や
なっ
れる
など
『
この
ない
-
』
ため
あっ
その
的
また
まで
よう
へ
られ
あり
なる
という
後
もの
第
"
中
だ
おり
により
これ
より
によって
,
）、
現在
による
）。
.
者
なり
だっ
ず
か
において
なかっ
られる
場合
時
行わ
その後
[日本]
多く
なく
について
名
せ
しかし
ば
間
できる
それ
上
使用
内
日本
化
他
作品
放送
当時
存在
数
呼ば
